In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Accuracy, Precision, Recall
from shutil import copyfile
import matplotlib.pyplot as plt
from selenium import webdriver
import urllib.request
import zipfile

In [ ]:


# DONOT CLOSE THE SELENEIUM AUTOMATED CHROME BROWSER
# Set the path to the chromedriver executable
# Replace with the path to your own chromedriver executable
chrome_path = 'path/to/chromedriver.exe'
driver = webdriver.Chrome(chrome_path)

# Navigate to the download page
driver.get('https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip')


local_zip = '/tmp/cats-and-dogs.zip'
urllib.request.urlretrieve('https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', local_zip)

# Extract the zip file
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Close the browser
driver.quit()

In [ ]:
source_path = '/tmp/PetImages'

source_path_dogs = os.path.join(source_path, 'Dog')
source_path_cats = os.path.join(source_path, 'Cat')


# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_dogs))} images of dogs.")
print(f"There are {len(os.listdir(source_path_cats))} images of cats.")



In [ ]:
# Define root directory
root_dir = '/tmp/cats-v-dogs'

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

# GRADED FUNCTION: create_train_test_dirs
def create_train_test_dirs(root_path):
  ### START CODE HERE
  path = os.path.join(root_path, "training")
  os.makedirs(path)
  path_1 = os.path.join(path, "cats")
  os.makedirs(path_1)
  path_2 = os.path.join(path, "dogs")
  os.makedirs(path_2)
  path = os.path.join(root_path, "testing")
  os.makedirs(path)
  path_3 = os.path.join(path, "cats")
  os.makedirs(path_3)
  path_4 = os.path.join(path, "dogs")
  os.makedirs(path_4)
  # HINT:
  # Use os.makedirs to create your directories with intermediate subdirectories

  

  ### END CODE HERE

  
try:
  create_train_test_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")


In [ ]:
# Test your create_train_test_dirs function

for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

In [ ]:
import os
import random
from shutil import copyfile

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    # Create the training and testing directories if they don't exist
    if not os.path.exists(TRAINING):
        os.makedirs(TRAINING)
    if not os.path.exists(TESTING):
        os.makedirs(TESTING)
    
    # Get all the files in the source directory
    files = []
    for filename in os.listdir(SOURCE):
        file = os.path.join(SOURCE, filename)
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + ' is zero length, so ignoring.')
    
    # Shuffle the files and split them into training and testing sets
    random.shuffle(files)
    training_set = files[:10000]
    testing_set = files[-3500:]
    
    # Copy the files into the training and testing directories
    for filename in training_set:
        src_file = os.path.join(SOURCE, filename)
        dest_file = os.path.join(TRAINING, filename)
        copyfile(src_file, dest_file)
    
    for filename in testing_set:
        src_file = os.path.join(SOURCE, filename)
        dest_file = os.path.join(TESTING, filename)
        copyfile(src_file, dest_file)
        
    print(f"{len(training_set)} images moved to {TRAINING}")
    print(f"{len(testing_set)} images moved to {TESTING}")


In [ ]:
# Test your split_data function

# Define paths
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"

TRAINING_DIR = "/tmp/cats-v-dogs/training/"
TESTING_DIR = "/tmp/cats-v-dogs/testing/"

TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
TESTING_CATS_DIR = os.path.join(TESTING_DIR, "cats/")

TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
TESTING_DOGS_DIR = os.path.join(TESTING_DIR, "dogs/")

# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_CATS_DIR)) > 0:
  for file in os.scandir(TRAINING_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_DOGS_DIR)) > 0:
  for file in os.scandir(TRAINING_DOGS_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_CATS_DIR)) > 0:
  for file in os.scandir(TESTING_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_DOGS_DIR)) > 0:
  for file in os.scandir(TESTING_DOGS_DIR):
    os.remove(file.path)

# Define proportion of images used for training
split_size = .9

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Check that the number of images matches the expected output
print(f"\n\nThere are {len(os.listdir(TRAINING_CATS_DIR))} images of cats for training")
print(f"There are {len(os.listdir(TRAINING_DOGS_DIR))} images of dogs for training")
print(f"There are {len(os.listdir(TESTING_CATS_DIR))} images of cats for testing")
print(f"There are {len(os.listdir(TESTING_DOGS_DIR))} images of dogs for testing")

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    train_datagen = ImageDataGenerator(rescale=1.0/255.)
    train_generator = train_datagen.flow_from_directory(
        directory=TRAINING_DIR,
        batch_size=150,
        class_mode='binary',
        target_size=(150, 150))
    
    validation_datagen = ImageDataGenerator(rescale=1.0/255.)
    validation_generator = validation_datagen.flow_from_directory(
        directory=VALIDATION_DIR,
        batch_size=150,
        class_mode='binary',
        target_size=(150, 150))
    
    return train_generator, validation_generator


train_generator, validation_generator = train_val_generators(TRAINING_DIR, TESTING_DIR)




In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

def create_model():
    binary_precision = tf.keras.metrics.Precision(name='binary_precision')
    binary_recall = tf.keras.metrics.Recall(name='binary_recall')

    model = tf.keras.models.Sequential([ 
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy', binary_precision, binary_recall])
    
    return model

model = create_model()

# add ModelCheckpoint callback to save the best model based on validation accuracy
callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator,
                    callbacks=callbacks)

# load the best saved model
best_model = tf.keras.models.load_model('best_model.h5')

# evaluate the best model
loss, accuracy, precision, recall = best_model.evaluate(validation_generator)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Load the saved model
best_model = tf.keras.models.load_model('best_model.h5')

# Define the path to the sample1 folder
folder_path = 'sample1/'

# Loop through all image files in the folder
for filename in os.listdir(folder_path):
    # Load the image and prepare it for prediction
    img_path = os.path.join(folder_path, filename)
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction using the loaded model
    prediction = best_model.predict(img_array)
    if prediction[0] < 0.5:
        print(f"{filename}: The image is a cat.")
    else:
        print(f"{filename}: The image is a dog.")


In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved model
best_model = tf.keras.models.load_model('best_model.h5')

# Define the function for making predictions on camera frames
def predict_camera():
    # Set up the video capture device (default webcam)
    cap = cv2.VideoCapture(0)

    while True:
        # Capture a frame from the camera
        ret, frame = cap.read()

        # Check if the frame was successfully captured
        if not ret:
            break

        # Preprocess the frame for prediction
        img = cv2.resize(frame, (150, 150))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        # Make a prediction using the loaded model
        prediction = best_model.predict(img_array)
        if prediction[0] < 0.5:
            label = "Cat"
        else:
            label = "Dog"

        # Display the label on the screen
        cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('frame', frame)

        # Wait for the user to press the "q" key to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture device and close the window
    cap.release()
    cv2.destroyAllWindows()

# Call the predict_camera function to start making predictions



In [ ]:
predict_camera()

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved model
best_model = tf.keras.models.load_model('best_model.h5')

# Define the function for making predictions on camera frames
def predict_camera():
    # Set up the video capture device (default webcam)
    cap = cv2.VideoCapture(1)

    while True:
        # Capture a frame from the camera
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame from camera")
            break

        # Preprocess the frame for prediction
        img = cv2.resize(frame, (150, 150))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        # Make a prediction using the loaded model
        prediction = best_model.predict(img_array)
        if prediction[0] < 0.5:
            label = "Cat"
        else:
            label = "Dog"

        # Display the label on the screen
        cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('frame', frame)

        # Wait for the user to press the "q" key to quit
        if cv2.waitKey(1) == ord('q'):
            break

    # Release the video capture device and close the window
    cap.release()
    cv2.destroyAllWindows()

# Call the predict_camera function to start making predictions
predict_camera()
